In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

In [2]:
data = pd.read_csv('data/retail_train.csv')
data.head(10)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0
5,2375,26984851516,1,826249,2,1.98,364,-0.60,1642,1,0.0,0.0
6,2375,26984851516,1,1043142,1,1.57,364,-0.68,1642,1,0.0,0.0
7,2375,26984851516,1,1085983,1,2.99,364,-0.40,1642,1,0.0,0.0
8,2375,26984851516,1,1102651,1,1.89,364,0.00,1642,1,0.0,0.0
9,2375,26984851516,1,6423775,1,2.00,364,-0.79,1642,1,0.0,0.0


In [3]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

### Задание 1. Weighted Random Recommendation

Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж
- Можно сэмплировать товары случайно, но пропорционально какому-либо весу
- Например, прямопропорционально популярности. Вес = log(sales_sum товара)

In [4]:
item_weights = data[['item_id', 'quantity']].groupby(['item_id']).sum().reset_index()
item_weights = item_weights[item_weights['quantity']>0]
item_weights['quantity'] = np.log(item_weights['quantity'])
item_weights['weight'] = item_weights['quantity'] / sum(item_weights['quantity'])
sum(item_weights['weight'])

1.0000000000002727

In [5]:
def weighted_random_recommendation(items_weights, n=5000):
    """Случайные рекоммендации
    
    Input
    -----
    items_weights: pd.DataFrame
        Датафрейм со столбцами item_id, weight. Сумма weight по всем товарам = 1
    """
    
    # Подсказка: необходимо модифицировать функцию random_recommendation()
    # your_code
    items = np.array(items_weights['item_id'])
    distribution = np.array(items_weights['weight'])  
    recs = np.random.choice(items, size=n, replace=False, p=distribution)
    
    return recs.tolist()

In [6]:
%%time

weighted_random_recommendation(item_weights)

CPU times: user 3.42 ms, sys: 0 ns, total: 3.42 ms
Wall time: 3.54 ms


[1080784,
 13008233,
 8118999,
 12949856,
 1082772,
 977347,
 1083548,
 835029,
 854257,
 12263494,
 8181028,
 5995830,
 15626016,
 946567,
 9677323,
 10457593,
 1842638,
 996712,
 1091342,
 242286,
 15972590,
 921422,
 967065,
 12604395,
 1781959,
 13777061,
 1082141,
 947113,
 1127097,
 10356021,
 871279,
 6565589,
 8069084,
 1077330,
 8019602,
 5568445,
 13097940,
 1901028,
 1000334,
 6633276,
 1067606,
 1060529,
 6961848,
 950998,
 1994074,
 846892,
 905539,
 999192,
 1090467,
 9297500,
 844278,
 1138487,
 9489380,
 1029894,
 851903,
 5567176,
 848402,
 6552920,
 970202,
 9651291,
 829368,
 868764,
 16122244,
 15972115,
 871063,
 1096509,
 15483691,
 9221532,
 1027474,
 9419525,
 5568721,
 1137641,
 852572,
 6602273,
 1035878,
 1989138,
 1108429,
 839461,
 9487278,
 15973012,
 9677371,
 5592218,
 987134,
 1095062,
 913270,
 6391330,
 837366,
 15593701,
 8205198,
 13094911,
 1120297,
 8160332,
 954006,
 974766,
 1028275,
 7441192,
 7144475,
 959544,
 887408,
 7166509,
 1114483,
 109

## Задание 2. Улучшение бейзлайнов и ItemItem

- Попробуйте улучшить бейзлайны, считая случаный на топ-5000 товаров
- Попробуйте улучшить разные варианты ItemItemRecommender, выбирая число соседей $K$.

In [7]:
# your_code

In [8]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']

#### 1.1 Random recommendation

In [9]:
def random_recommendation(items, n=5):
    """Случайные рекоммендации"""
    
    items = np.array(items)
    recs = np.random.choice(items, size=n, replace=False)
    
    return recs.tolist()

In [10]:
items = data_train.item_id.unique()
result['random_recommendation'] = result['user_id'].apply(lambda x: random_recommendation(items, n=500))

#### 1.2 Popularity-based recommendation

In [11]:
def popularity_recommendation(data, n=5):
    """Топ-n популярных товаров"""
    
    popular = data.groupby('item_id')['sales_value'].sum().reset_index()
    popular.sort_values('sales_value', ascending=False, inplace=True)
    
    recs = popular.head(n).item_id
    
    return recs.tolist()

In [12]:
popular_recs = popularity_recommendation(data_train, n=5)
result['popular_recommendation'] = result['user_id'].apply(lambda x: popular_recs)

#### 1.3 Weighted random recommender

In [13]:
weighted_recs = weighted_random_recommendation(item_weights, n=500)
result['weighted_random_recommendation'] = result['user_id'].apply(lambda x: weighted_recs)

### Results

In [14]:
def precision(recommended_list, bought_list):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    flags = np.isin(bought_list, recommended_list)
    return flags.sum() / len(recommended_list)

In [15]:
def precision_at_k(recommended_list, bought_list, k=5):
    return precision(recommended_list[:k], bought_list)

In [16]:
result

,user_id,actual,random_recommendation,popular_recommendation,weighted_random_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[1087755, 835976, 894357, 850430, 1004852, 103...","[6534178, 6533889, 1029743, 6534166, 1082185]","[12604978, 915390, 880000, 828516, 1454055, 92..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[879528, 15452726, 1896368, 965009, 1158791, 1...","[6534178, 6533889, 1029743, 6534166, 1082185]","[12604978, 915390, 880000, 828516, 1454055, 92..."
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[5646038, 1076889, 984671, 9527260, 8205651, 8...","[6534178, 6533889, 1029743, 6534166, 1082185]","[12604978, 915390, 880000, 828516, 1454055, 92..."
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[5569148, 9487727, 820918, 5564122, 865775, 86...","[6534178, 6533889, 1029743, 6534166, 1082185]","[12604978, 915390, 880000, 828516, 1454055, 92..."
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[1108296, 6464201, 12605694, 1112754, 6551930,...","[6534178, 6533889, 1029743, 6534166, 1082185]","[12604978, 915390, 880000, 828516, 1454055, 92..."
...,...,...,...,...,...
2037,2496,[6534178],"[16828194, 9485210, 1024211, 2434629, 1010164,...","[6534178, 6533889, 1029743, 6534166, 1082185]","[12604978, 915390, 880000, 828516, 1454055, 92..."
2038,2497,"[1016709, 9835695, 1132298, 16809501, 845294, ...","[853822, 1035097, 1071538, 10455642, 12385999,...","[6534178, 6533889, 1029743, 6534166, 1082185]","[12604978, 915390, 880000, 828516, 1454055, 92..."
2039,2498,"[15716530, 834484, 901776, 914190, 958382, 972...","[1111129, 8014569, 1077699, 1001502, 945540, 3...","[6534178, 6533889, 1029743, 6534166, 1082185]","[12604978, 915390, 880000, 828516, 1454055, 92..."
2040,2499,"[867188, 877580, 902396, 914190, 951590, 95813...","[1264020, 6391216, 3689063, 12810457, 974688, ...","[6534178, 6533889, 1029743, 6534166, 1082185]","[12604978, 915390, 880000, 828516, 1454055, 92..."


In [17]:
for name_col in result.columns[1:]:
    print(f"{round(result.apply(lambda row: precision_at_k(row[name_col], row['actual']), axis=1).mean(),15)}:{name_col}")

1.0:actual
0.00048971596474:random_recommendation
0.155239960822723:popular_recommendation
0.001469147894221:weighted_random_recommendation
